In [1]:
from pathlib import Path
from types import SimpleNamespace
import warnings
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from typing import Optional, List, Callable, Tuple
from torchrl.modules import MLP
from Transformer_model import myGlue
pruning_keypoint_thresholds = 1024
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

%load_ext autoreload
%autoreload 2

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 'mps', 'cpu'
print(torch.cuda.is_available())

True


In [8]:
x = torch.load('/home/koki/LightGlue/scannet_dataset/scannet_embed/scene0000_00.pth')
print(x[0].keys())
print(len(x))

dict_keys(['keypoints', 'keypoint_scores', 'descriptors', 'image_size'])
5578


In [11]:
x[0]['descriptors'].shape
x[1]['descriptors'].shape

torch.Size([1, 2048, 3])

In [16]:
import torch

# Create a sample tensor
tensor = torch.randn(4, 2048, 3)

# Split the tensor into 4 tensors
split_tensors = torch.split(tensor, 1, dim=0)
print(split_tensors[0].shape)
# Convert the split_tensors to a list of tensors (if needed)
split_tensors_list = list(split_tensors)
data = []
data.extend(split_tensors_list)
print('=================  ',data[0].shape)
# Print the shapes of the split tensors
for i, split_tensor in enumerate(split_tensors_list):
    print(f"Split Tensor {i + 1} shape: {split_tensor.shape}")

torch.Size([1, 2048, 3])
=================   torch.Size([1, 2048, 3])
Split Tensor 1 shape: torch.Size([1, 2048, 3])
Split Tensor 2 shape: torch.Size([1, 2048, 3])
Split Tensor 3 shape: torch.Size([1, 2048, 3])
Split Tensor 4 shape: torch.Size([1, 2048, 3])


In [5]:
transformers = myGlue()
print(transformers)

myGlue(
  (input_proj): Linear(in_features=3, out_features=3, bias=True)
  (transformers): ModuleList(
    (0): MyTransformerLayer(
      (self_attn): SelfBlock(
        (toqkv): Linear(in_features=3, out_features=9, bias=True)
        (out_proj): Linear(in_features=3, out_features=3, bias=True)
        (inner_attn): Attention()
      )
      (cross_attn): CrossBlock()
    )
    (1): MyTransformerLayer(
      (self_attn): SelfBlock(
        (toqkv): Linear(in_features=16, out_features=48, bias=True)
        (out_proj): Linear(in_features=16, out_features=16, bias=True)
        (inner_attn): Attention()
      )
      (cross_attn): CrossBlock()
    )
    (2): MyTransformerLayer(
      (self_attn): SelfBlock(
        (toqkv): Linear(in_features=32, out_features=96, bias=True)
        (out_proj): Linear(in_features=32, out_features=32, bias=True)
        (inner_attn): Attention()
      )
      (cross_attn): CrossBlock()
    )
    (3): MyTransformerLayer(
      (self_attn): SelfBlock(
     

In [24]:
class MatchAssignment(nn.Module):
    def __init__(self,dim: int):
        super().__init__()
        self.dim = dim
        self.matchability = nn.Linear(dim, 1, bias=True)
        self.final_proj = nn.Linear(dim, dim, bias=True)
    def forward(self, desc0: torch.Tensor, desc1: torch.Tensor):
        mdesc0, mdesc1 = self.final_proj(desc0), self.final_proj(desc1)
        _, _, d = mdesc0.shape
        mdesc0, mdesc1 = mdesc0 / d**.25, mdesc1 / d**.25
        sim = torch.einsum('bmd,bnd->bmn', mdesc0, mdesc1)
        return sim

In [36]:
gg = MatchAssignment(2)
desc0 = torch.randn(1,4,2)
desc1 = torch.randn(1,3,2)

sim = gg(desc0,desc1)
print(sim)

tensor([[[-0.1343,  0.3721, -0.4150],
         [ 0.3869, -0.4664,  0.7231],
         [ 0.0855,  0.0220,  0.0622],
         [ 0.1266, -0.1156,  0.2077]]], grad_fn=<ViewBackward0>)


In [4]:
class Attention(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.has_sdp = hasattr(F, 'scaled_dot_product_attention')

    def forward(self, q, k, v, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        if self.has_sdp:
            args = [x.contiguous() for x in [q, k, v]]
            v = F.scaled_dot_product_attention(*args, attn_mask=mask)
            return v if mask is None else v.nan_to_num()

In [8]:
# Optionally use the context manager to ensure one of the fused kernels is run
query = torch.rand(32, 8, 128, 64, dtype=torch.float16, device="cuda")
key = torch.rand(32, 8, 128, 64, dtype=torch.float16, device="cuda")
value = torch.rand(32, 8, 128, 128, dtype=torch.float16, device="cuda")
with torch.backends.cuda.sdp_kernel(enable_math=False):
    x = F.scaled_dot_product_attention(query, key, value)
print(x.shape)

torch.Size([32, 8, 128, 128])
